# Knapsack Problem - Genetic Algorithms


`Carlos Alfonso Hinojosa Cavada | A01137566`

`Miguel Angel Cortes Guzman | A01270966`

`Jesus Eider Diaz Moraila | A00828174`

## Representation

In [31]:
import random
import numpy
import pandas as pd
import os

# Implements the random initialization of individuals using the binary representation.
def createIndividual(nbBits):
  return numpy.random.permutation(nbBits)

## Selection

In [3]:
# Implements the tournament selection.
def select(population, evaluation, tournamentSize):
  winner = numpy.random.randint(0, len(population))
  for i in range(tournamentSize - 1):
    rival = numpy.random.randint(0, len(population))
    if (evaluation[rival] < evaluation[winner]):
      winner = rival
  return population[winner]

## Crossover

In [58]:
# Implements the one point crossover on individuals using the binary representation.
def combine(parentA, parentB, cRate):
  if (random.random() <= cRate):
    cPoint = numpy.random.randint(1, len(parentA))   
    offspringA = numpy.append(parentA[0:cPoint], parentB)
    _, idx = numpy.unique(offspringA, return_index=True)
    offspringA = offspringA[numpy.sort(idx)]

    offspringB = numpy.append(parentB[0:cPoint], parentA)
    _, idx = numpy.unique(offspringB, return_index=True)
    offspringB = offspringB[numpy.sort(idx)]
  else:
    offspringA = numpy.copy(parentA)
    offspringB = numpy.copy(parentB)
  return offspringA, offspringB

## Mutation

In [67]:
# Implements the flip mutation on individuals using the binary representation.
def mutate(individual, mRate):
  idx1 = random.randint(0, len(individual)-1)
  idx2 = random.randint(0, len(individual)-1)
  if (random.random() <= mRate):
      individual[idx1], individual[idx2] = individual[idx2], individual[idx1]  
  return individual

## Evaluation

In [33]:
# Implements the fitness function of individuals using the binary representation and solving the balanced partition set.
def evaluate(individual, problem, capacity):
    value = 0
    weight = 0

    # Calculate the weight and value of current solution
    for i in range(len(individual)):
        item_weight = problem.iloc[i, 0]
        item_value = problem.iloc[i, 1]

        if weight + item_weight > capacity:
            break
        else:
            weight += item_weight
            value += item_value

    return value

In [95]:
# Implements the fitness function of individuals using the binary representation and solving the balanced partition set.
def print_solution(individual, problem, capacity):
    weight = 0
    items = []

    # Calculate the weight and value of current solution
    for i in range(len(individual)):
        item_weight = problem.iloc[i, 0]

        if weight + item_weight > capacity:
            return len(items)
            break
        else:
            items.append(individual[i])
            weight += item_weight

## Genetic Algorithm

In [7]:
# Implements a genetic algorithm for solving the balanced partition problem with individuals using the binary representation.
def geneticAlgorithm(problem, populationSize, generations, cRate, mRate, capacity):
  # Creates the initial population (it also evaluates it)
  population = [None] * populationSize
  evaluation = [None] * populationSize  
  for i in range(populationSize):
    individual = createIndividual(len(problem))
    population[i] = individual
    evaluation[i] = evaluate(individual, problem, capacity)
  # Keeps a record of the best individual found so far
  index = 0
  for i in range(1, populationSize):
    if (evaluation[i] > evaluation[index]):
      index = i
  bestIndividual = population[index]
  bestEvaluation = evaluation[index]
  
  # Runs the evolutionary process    
  for i in range(generations):
    k = 0
    newPopulation = [None] * populationSize    
    for j in range(populationSize // 2):
      parentA = select(population, evaluation, 3)
      parentB = select(population, evaluation, 3)
      newPopulation[k], newPopulation[k + 1] = combine(parentA, parentB, cRate)       
      k = k + 2    
    population = newPopulation
    for i in range(populationSize):
      population[i] = mutate(population[i], mRate)
      evaluation[i] = evaluate(population[i], problem, capacity)
      # Keeps a record of the best individual found so far
      if (evaluation[i] > bestEvaluation):
        bestEvaluation = evaluation[i]
        bestIndividual = population[i]
  return bestIndividual, bestEvaluation

## Driver Program

In [97]:
# Before running the code, we must initialize the random number generators.
numpy.random.seed(42)
random.seed(numpy.random.randint(10000))
instance_dir = ".\Knapsack Problems\\" # Write your local KS instance directory
instances = os.listdir(instance_dir)

for instance in instances:
    df = pd.read_csv(instance_dir + instance, header = None)
    df.head()

    # Define parameters for genetic algorithm
    problem, header = df.drop(df.head(1).index),df.head(1)
    capacity = int(header[1])
    populationSize = 20
    generations = 25
    cRate = 0.9
    mRate = 0.01

    # Solve the problem
    solution, evaluation = geneticAlgorithm(problem, populationSize, generations, cRate, mRate, capacity)
    # print("Solution")
    # print(solution)
    num_items = print_solution(solution, problem, capacity)
    print("Instance: " + instance)
    print("Evaluation: " + str(evaluation))
    print("Total items in knapsack: " + str(num_items))
    print("Solution: " + str(solution[0:num_items]))
    print("---------------------------------------")


Instance: ks_10000_0
Evaluation: 937157.0
Total items in knapsack: 9
Solution: [6252 4684 1731 4742 4521 6340  576 5202 6363]
---------------------------------------
Instance: ks_1000_0
Evaluation: 100890.0
Total items in knapsack: 10
Solution: [173 745 849 266 391 106 498 973 651 669]
---------------------------------------
Instance: ks_100_0
Evaluation: 90000.0
Total items in knapsack: 1
Solution: [31]
---------------------------------------
Instance: ks_100_1
Evaluation: 1230813.0
Total items in knapsack: 36
Solution: [23 12 22 64 85 24 44 83 98 36 51 17 29 48 52 95 56 28 78 25 18 68 41 46
 92 63 70 47  8 45  6  1 38 21 53 65]
---------------------------------------
Instance: ks_100_2
Evaluation: 9682.0
Total items in knapsack: 9
Solution: [54 40 11 60 99 79 94  4 24]
---------------------------------------
Instance: ks_106_0
Evaluation: 92679972.0
Total items in knapsack: 11
Solution: [ 22  26   3  38  62 102  40 101  31  61  11]
---------------------------------------
Instance: ks